In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px  # (version 4.7.0 or higher)
import plotly.graph_objects as go
#from dash import Dash
from jupyter_dash import JupyterDash
#from dash import dcc
import dash_core_components as dcc 
#from dash import html
import dash_html_components as html
from dash.dependencies import Input, Output

In [2]:
df=pd.DataFrame()
df['time']=np.arange(0,2000,5)
df['System ON in Auto Mode']= np.random.randint(0,2,400)
df['Operating Mode']= np.random.randint(0,2,400)
df['Billet Size']= np.random.randint(120,121,400)
df['Soaking Zone Temp']= np.random.normal(1185,2,400)
df['Heating Zone Temp']= np.random.normal(985,2,400)
df['Furnace pressure']= np.random.normal(0.8,1.5,400)
df['Oxygen %']= np.random.normal(0.02,1.5,400)
df['Soaking Zone Set Point']= np.random.randint(1185,1186,400)
df['Heating Zone Set Point']= np.random.randint(985,986,400)
df['Furnace pressure Set Point']= np.random.random(400)
df['Oxygen % Set Point']= np.random.randint(2,3,400)
df['Soaking Zone Coal Motor RPM']= np.random.randint(1175,1491,400)
df['Air Motor RPM']= np.random.randint(1200,1476,400)
df['Pressure PID  Output %']= np.random.randint(95,96,400)
df['Furnace Damper Opening']= np.random.randint(95,96,400)
df['Flue Gas temp before Rec']= np.random.randint(712,752,400)
df['Flue Gas temp aftre Rec']= np.random.randint(255,290,400)
df['Combustion Air temp before Rec']= np.random.randint(45,52,400)
df['Combustion Air temp After Rec']= np.random.randint(325,350,400)
df['Combustion Air Pressure Before Rec']= np.random.randint(1475,1486,400)
df['Combustion Air Pressure After Rec']= np.random.randint(1478,1501,400)
df['Coal Motor1 RPM set point Min']= np.random.randint(1175,1176,400)
df['Coal Motor1 RPM set point Max']= np.random.randint(1430,1431,400)
df['Air Motor RPM set point Min']= np.random.randint(1200,1201,400)
df['Air Motor RPM set point Max']= np.random.randint(1475,1476,400)
df['Furnace Damper Opening Set Min']= np.random.randint(50,51,400)
df['Furnace Damper Opening Set Max']= np.random.randint(95,96,400)
df['Air Motor Current ']= np.random.randint(35,43,400)
df['Coal Consumption (Cumulative-Sum)']= np.linspace(55,55+399,num=400)

In [12]:
df.to_csv('fykyas_drive.csv')

In [3]:
X = df[['Soaking Zone Temp', 'Heating Zone Temp', 'Furnace pressure', 'Soaking Zone Coal Motor RPM','Oxygen %','Air Motor RPM','Combustion Air temp before Rec','Combustion Air temp After Rec','Combustion Air Pressure Before Rec','Combustion Air Pressure After Rec']]
print(X.shape)
print(X.columns)
corr_df= X.corr()
print(corr_df.shape)
print((X.corrwith(X['Soaking Zone Temp'])))


(400, 10)
Index(['Soaking Zone Temp', 'Heating Zone Temp', 'Furnace pressure',
       'Soaking Zone Coal Motor RPM', 'Oxygen %', 'Air Motor RPM',
       'Combustion Air temp before Rec', 'Combustion Air temp After Rec',
       'Combustion Air Pressure Before Rec',
       'Combustion Air Pressure After Rec'],
      dtype='object')
(10, 10)
Soaking Zone Temp                     1.000000
Heating Zone Temp                     0.008033
Furnace pressure                     -0.006420
Soaking Zone Coal Motor RPM           0.069216
Oxygen %                              0.032491
Air Motor RPM                        -0.039356
Combustion Air temp before Rec       -0.050743
Combustion Air temp After Rec         0.090362
Combustion Air Pressure Before Rec   -0.012085
Combustion Air Pressure After Rec     0.017808
dtype: float64


In [4]:
c=pd.DataFrame(corr_df.columns.values.tolist(),columns=['column_name'])

In [5]:
c.columns.values

array(['column_name'], dtype=object)

In [6]:
print(X.corr())

                                    Soaking Zone Temp  Heating Zone Temp  \
Soaking Zone Temp                            1.000000           0.008033   
Heating Zone Temp                            0.008033           1.000000   
Furnace pressure                            -0.006420           0.078561   
Soaking Zone Coal Motor RPM                  0.069216           0.068344   
Oxygen %                                     0.032491           0.025204   
Air Motor RPM                               -0.039356          -0.053445   
Combustion Air temp before Rec              -0.050743          -0.002661   
Combustion Air temp After Rec                0.090362          -0.105991   
Combustion Air Pressure Before Rec          -0.012085          -0.017834   
Combustion Air Pressure After Rec            0.017808          -0.024655   

                                    Furnace pressure  \
Soaking Zone Temp                          -0.006420   
Heating Zone Temp                           0.07856

In [7]:
app = JupyterDash(__name__)

In [8]:
app.layout= html.Div(children=[
                      html.H1("Dashboards with Dash",style={'text-align':'center'}),
                      dcc.Dropdown(id='select_x',
                                   options=[{'label':i, 'value':i} for i in X.columns.values],
                                   value= corr_df.columns.values[0],#col1, #x2_df#x2_values.values(),#corr_df.columns.to_list(),
                                   style={"width":"50%"}
                                   ),
                      #html.Div(id='output_container',children=[]),
                      html.Br(),
                      dcc.Graph(id='correlation_bar_graph',figure={})
])

In [9]:
# Connect the Plotly graphs with Dash Components
@app.callback(

    #Output(component_id='output_container', component_property='children'),
    Output(component_id='correlation_bar_graph', component_property='figure'),
    [Input(component_id='select_x', component_property='value')]
)
def update_graph(value_chosen):
    
    dff = corr_df#.iloc[value_chosen]
    #dff['column_name']=dff.columns.values
    data = {'x':dff.columns.values,'y':dff[value_chosen]}
    #[20, 21, 19, 18]}
    #cols = dff["column_name"] == value_chosen
        
    figure = px.bar(
          data_frame=data, #corr_df,
          x="x",#dff.index.values.tolist(),
          y="y",#yy,
          
          template="plotly_dark"
    )
    #figure.update_xaxes(type='category')
    figure.update_layout(xaxis_type='category')
    return (figure)

In [11]:
app.run_server(mode='external', debug=False, port=8052)


 * Running on http://127.0.0.1:8052/ (Press CTRL+C to quit)
127.0.0.1 - - [25/May/2022 15:20:01] "GET /_alive_e9c09be6-3090-4e8f-bc8b-34536d3612db HTTP/1.1" 200 -


Dash app running on http://127.0.0.1:8052/


127.0.0.1 - - [25/May/2022 15:20:16] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2022 15:20:16] "GET /_dash-component-suites/dash_renderer/polyfill@7.v1_9_0m1653279883.8.7.min.js HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2022 15:20:16] "GET /_dash-component-suites/dash_renderer/react@16.v1_9_0m1653279883.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2022 15:20:16] "GET /_dash-component-suites/dash_renderer/prop-types@15.v1_9_0m1653279883.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2022 15:20:16] "GET /_dash-component-suites/dash_renderer/react-dom@16.v1_9_0m1653279883.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2022 15:20:16] "GET /_dash-component-suites/dash_html_components/dash_html_components.v1_1_2m1653279884.min.js HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2022 15:20:16] "GET /_dash-component-suites/dash_core_components/dash_core_components-shared.v1_15_0m1653279884.js HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2022 15:20:16] "GET /_dash-component-suites/dash_core_components/da